In [1]:
%%capture
pip install pandas 
pip install geopandas 
pip install requests 

SyntaxError: invalid syntax (<ipython-input-1-c75d43f3af4c>, line 1)

In [1]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

/opt/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
shp_100 = gpd.read_file("/Users/panssies/Desktop/diplom/practice/lermontov/buf_100.gpkg", encoding='latin1')
shp_250 = gpd.read_file("/Users/panssies/Desktop/diplom/practice/lermontov/buf_250.gpkg", encoding='latin1')
shp_250.head()

,DN,id,geometry
0,684,1,"MULTIPOLYGON (((42.97965 44.12672, 42.97962 44..."
1,685,2,"MULTIPOLYGON (((42.98007 44.12672, 42.98004 44..."
2,686,3,"MULTIPOLYGON (((42.98035 44.12672, 42.98032 44..."
3,687,4,"MULTIPOLYGON (((42.98062 44.12672, 42.98060 44..."
4,686,5,"MULTIPOLYGON (((42.98090 44.12672, 42.98087 44..."


In [3]:
shp_100.describe()

,DN,id
count,32915.000000,32915.000000
mean,814.063649,16458.000000
std,159.172976,9501.886392
min,598.000000,1.000000
25%,683.000000,8229.500000
50%,767.000000,16458.000000
75%,910.000000,24686.500000
max,1385.000000,32915.000000


In [4]:
shp_250.describe()

,DN,id
count,32915.000000,32915.000000
mean,814.063649,16458.000000
std,159.172976,9501.886392
min,598.000000,1.000000
25%,683.000000,8229.500000
50%,767.000000,16458.000000
75%,910.000000,24686.500000
max,1385.000000,32915.000000


In [5]:
# extract the geometry in GeoJSON format
geoms_100 = shp_100.geometry.values # list of shapely geometries
geoms_250 = shp_250.geometry.values

In [6]:
# transform to GeJSON format
from shapely.geometry import mapping
geoms_100 = [mapping(elem) for elem in geoms_100 if elem is not None]
geoms_250 = [mapping(elem) for elem in geoms_250 if elem is not None]
#geoms = res_geom
res_geoms_100 = []
res_geoms_250 = []

In [7]:
shp_100["heights_diff"] = np.nan
with rasterio.open("/Users/panssies/Desktop/diplom/practice/lermontov/DEM_lermontov_30m.tif") as src:
    for index, cur_geom in tqdm(enumerate(geoms_100)):
        out_image_100, out_transform = mask(src, [cur_geom], crop=True)
        out_image_100 = out_image_100[0]
        tmp_need_100 = out_image_100[np.where(~np.isclose(out_image_100, np.min(out_image_100)))]
        shp_100.at[index, 'dispersion'] = np.var(tmp_need_100)
        if len(tmp_need_100) > 0:
            shp_100.at[index, 'heights_diff'] = np.max(tmp_need_100) - np.min(tmp_need_100)
        else:
            shp_100.at[index, 'heights_diff'] = 0
        shp_100.at[index, 'median'] = np.median((tmp_need_100))
    print(shp_100)
    
    

32915it [00:11, 2978.01it/s]


        DN     id                                           geometry  \
0      684      1  MULTIPOLYGON (((42.97778 44.12669, 42.97777 44...   
1      685      2  MULTIPOLYGON (((42.97819 44.12669, 42.97818 44...   
2      686      3  MULTIPOLYGON (((42.97847 44.12669, 42.97846 44...   
3      687      4  MULTIPOLYGON (((42.97875 44.12669, 42.97874 44...   
4      686      5  MULTIPOLYGON (((42.97903 44.12669, 42.97902 44...   
...    ...    ...                                                ...   
32910  656  32911  MULTIPOLYGON (((43.02153 44.06974, 43.02152 44...   
32911  653  32912  MULTIPOLYGON (((43.02180 44.06974, 43.02179 44...   
32912  651  32913  MULTIPOLYGON (((43.02208 44.06974, 43.02207 44...   
32913  650  32914  MULTIPOLYGON (((43.02236 44.06974, 43.02235 44...   
32914  649  32915  MULTIPOLYGON (((43.02264 44.06974, 43.02263 44...   

       heights_diff  dispersion  median  
0              15.0   22.806400   690.0  
1              15.0   21.715976   689.0  
2        

In [8]:
shp_250.head()

,DN,id,geometry
0,684,1,"MULTIPOLYGON (((42.97965 44.12672, 42.97962 44..."
1,685,2,"MULTIPOLYGON (((42.98007 44.12672, 42.98004 44..."
2,686,3,"MULTIPOLYGON (((42.98035 44.12672, 42.98032 44..."
3,687,4,"MULTIPOLYGON (((42.98062 44.12672, 42.98060 44..."
4,686,5,"MULTIPOLYGON (((42.98090 44.12672, 42.98087 44..."


In [9]:
shp_250["heights_diff"] = np.nan
with rasterio.open("/Users/panssies/Desktop/diplom/practice/lermontov/DEM_lermontov_30m.tif") as src:
    for index, cur_geom in tqdm(enumerate(geoms_250)):
        out_image_250, out_transform = mask(src, [cur_geom], crop=True)
        out_image_250 = out_image_250[0]
        tmp_need_250 = out_image_250[np.where(~np.isclose(out_image_250, np.min(out_image_250)))]
        shp_250.at[index, 'dispersion'] = np.var(tmp_need_250)
        if len(tmp_need_250) > 0:
            shp_250.at[index, 'heights_diff'] = np.max(tmp_need_250) - np.min(tmp_need_250)
        else:
            shp_250.at[index, 'heights_diff'] = 0
        shp_250.at[index, 'median'] = np.median((tmp_need_250))
    print(shp_250)

32915it [00:11, 2930.30it/s]


        DN     id                                           geometry  \
0      684      1  MULTIPOLYGON (((42.97965 44.12672, 42.97962 44...   
1      685      2  MULTIPOLYGON (((42.98007 44.12672, 42.98004 44...   
2      686      3  MULTIPOLYGON (((42.98035 44.12672, 42.98032 44...   
3      687      4  MULTIPOLYGON (((42.98062 44.12672, 42.98060 44...   
4      686      5  MULTIPOLYGON (((42.98090 44.12672, 42.98087 44...   
...    ...    ...                                                ...   
32910  656  32911  MULTIPOLYGON (((43.02340 44.06978, 43.02337 44...   
32911  653  32912  MULTIPOLYGON (((43.02368 44.06978, 43.02365 44...   
32912  651  32913  MULTIPOLYGON (((43.02395 44.06978, 43.02393 44...   
32913  650  32914  MULTIPOLYGON (((43.02423 44.06978, 43.02420 44...   
32914  649  32915  MULTIPOLYGON (((43.02451 44.06978, 43.02448 44...   

       heights_diff  dispersion  median  
0              28.0   52.636705   698.5  
1              28.0   54.914304   697.0  
2        

In [10]:
shp_250.head()

,DN,id,geometry,heights_diff,dispersion,median
0,684,1,"MULTIPOLYGON (((42.97965 44.12672, 42.97962 44...",28.0,52.636705,698.5
1,685,2,"MULTIPOLYGON (((42.98007 44.12672, 42.98004 44...",28.0,54.914304,697.0
2,686,3,"MULTIPOLYGON (((42.98035 44.12672, 42.98032 44...",28.0,55.496790,695.0
3,687,4,"MULTIPOLYGON (((42.98062 44.12672, 42.98060 44...",28.0,56.039407,695.0
4,686,5,"MULTIPOLYGON (((42.98090 44.12672, 42.98087 44...",28.0,56.553751,694.0


In [135]:
shp_100.to_file("/Users/panssies/Desktop/diplom/practice/lermontov/result100.shp")
shp_250.to_file("/Users/panssies/Desktop/diplom/practice/lermontov/result250.shp")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until
